### Importing library and creating spark-session

In [ ]:
import sys
fileDir = "/home/jovyan/notebooks/"
sys.path.append(fileDir)

from utilities import *
import pyspark.sql.functions as F

In [ ]:
session = create_spark_session("Neo4j GitHub", SparkConnector.NEO4J)

In [ ]:
common_URI = "hdfs://namenode:9000//data-team/"

### Reading from HDFS

In [ ]:
files = session.read.json(f"{common_URI}files.json")
files.printSchema()

In [ ]:
languages = session.read.json(f"{common_URI}languages.json")
languages.printSchema()

In [ ]:
licences = session.read.json(f"{common_URI}licenses.json")
licences.printSchema()

In [ ]:
commits = session.read.json(f"{common_URI}commits_min.json") # cambia se ti serve il dataset completo
commits.printSchema()

In [ ]:
git_commits= commits.select("commit", "subject", "message", "committer.date", "author.date")

In [ ]:
newColumns = ["id","title","message","committer_date", "author_date"]
git_commits=git_commits.toDF(*newColumns)
git_commits.printSchema()

In [ ]:
repositories = session.read.json(f"{common_URI}repositories_min.json") # cambia se ti serve il dataset completo
repositories.printSchema()

In [ ]:
git_repositories = repositories.withColumnRenamed("repo_name", "name")
git_repositories.printSchema()

In [ ]:
git_languages = languages.withColumn("name",F.explode(languages["language.name"])).dropDuplicates(["name"]).select("name")
git_languages.printSchema()

In [ ]:
git_licenses = licences.select("license").withColumnRenamed("license","name")
git_licenses.printSchema()

In [ ]:
git_files = files.select("id","ref","path","mode","symlink_target")
git_files.printSchema()

### Getting a mini-batch of the dataset

In [ ]:
repositories_min = repositories.sort("repo_name")
repositories_min = repositories_min.limit(100)
repositories_min.printSchema()
repositories_min.write.format("json").mode("overwrite").save(f"{common_URI}repositories_min.json")

In [ ]:
commits_min = commits.join(repositories_min, commits["repo"]==repositories_min["repo_name"]).withColumnRenamed("repo_name","repo2").drop("repo_name").drop("repo2")
commits_min.printSchema()

In [ ]:
commits_min.write.format("json").mode(
    "overwrite").save(f"{common_URI}commits_min.json")

In [ ]:
languages_min = languages.join(repositories_min, languages["repo_name"] == repositories_min["repo_name"]).select("language","name")

In [ ]:
languages_min.write.format("json").mode(
    "overwrite").save(f"{common_URI}languages_min.json")

In [ ]:
licenses_min = licences.join(repositories_min, licences["repo_name"] == repositories_min["repo_name"]).select("license","name")

In [ ]:
licenses_min.write.format("json").mode(
    "overwrite").save(f"{common_URI}licenses_min.json")

In [ ]:
files_min = files.join(repositories_min, files["repo_name"] == repositories_min["repo_name"]).select("id","ref","path","mode","symlink_target","name")

In [ ]:
files_min.write.format("json").mode(
    "overwrite").save(f"{common_URI}files_min.json")

### Writing the nodes in the graph

In [ ]:
"""options = get_default_options(SparkConnector.NEO4J)

options["node.keys"] = "name"
options["labels"] = ":GitLanguage"

spark_write(SparkConnector.NEO4J, git_languages, "Overwrite", options=options)"""

In [ ]:
"""options = get_default_options(SparkConnector.NEO4J)

options["node.keys"] = "name"
options["labels"] = ":GitLicense"

spark_write(SparkConnector.NEO4J, git_licenses, "Overwrite", options=options)"""

In [ ]:
"""options = get_default_options(SparkConnector.NEO4J)

options["node.keys"] = "id"
options["labels"] = ":GitFile"

spark_write(SparkConnector.NEO4J, git_files, "Overwrite", options=options)"""

In [9]:
options = get_default_options(SparkConnector.NEO4J)

options["node.keys"] = "id"
options["labels"] = ":GitCommit"

spark_write(SparkConnector.NEO4J, git_commits, "Overwrite", options=options)

In [ ]:
options = get_default_options(SparkConnector.NEO4J)

options["node.keys"] = "name"
options["labels"] = ":GitRepository"

spark_write(SparkConnector.NEO4J, git_repositories, "Overwrite", options=options)

### Writing the relationships in the graph

In [ ]:
commits_min.printSchema()

In [ ]:
rel = commits_min.select("id","repo_name")

options = get_default_options(SparkConnector.NEO4J)
options["relationship"] = "BELONGS_TO"
options["relationship.source.labels"] = "GitCommit"
options["relationship.save.strategy"] = "keys"
options["relationship.source.node.keys"] = "from:id"
options["relationship.target.labels"] = "GitRepository"
options["relationship.target.node.keys"] = "to:repo_name"

spark_write(SparkConnector.NEO4J, rel, "Overwrite", options=options)